In [2]:
library(ggplot2)
library(dplyr)
library(readr)
library(tidyr)
library(lmPerm)

In [3]:
df <- read_csv('../data/clean_outliers.csv')

Rows: 253680 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (22): Diabetes_012, HighBP, HighChol, CholCheck, BMI, Smoker, Stroke, He...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
# kiểm định chi-square với monte carlo simulation
chisq_test <- function(df, var1, var2) {
  observed <- table(df[[var1]], df[[var2]])
  chisq <- chisq.test(observed, simulate.p.value = TRUE, B = 10000)
  return(chisq)
}

In [5]:
# kiểm định kruskal-wallis
kruskal_test <- function(df, var1, var2) {
  kruskal <- kruskal.test(df[[var1]] ~ df[[var2]], data = df)
  return(kruskal)
}

#### Biến giải thích với predictor

In [7]:
df_category <- df %>% select(-c('BMI', 'MentHlth', 'PhysHlth', 'Diabetes_012'))

# lấy tên tất cả các cột trong df_category
vars <- colnames(df_category)

chi_square_results <- data.frame(variable = character(), p_value = numeric(), reject = logical(), stringsAsFactors = FALSE)

for (var in vars) {
    chi_square <- chisq_test(df, 'Diabetes_012', var)
    reject <- chi_square$p.value < 0.05
    chi_square_results <- rbind(chi_square_results, data.frame(variable = var, p_value = chi_square$p.value, reject = reject))
}

chi_square_results


variable,p_value,reject
<chr>,<dbl>,<lgl>
HighBP,9.999e-05,TRUE
HighChol,9.999e-05,TRUE
CholCheck,9.999e-05,TRUE
Smoker,9.999e-05,TRUE
Stroke,9.999e-05,TRUE
HeartDiseaseorAttack,9.999e-05,TRUE
PhysActivity,9.999e-05,TRUE
Fruits,9.999e-05,TRUE
Veggies,9.999e-05,TRUE


In [9]:
vars_quantitative <- c('BMI', 'PhysHlth', 'MentHlth')

for (var in vars_quantitative) {
   results <- aov(formula=df[[var]] ~ df[['Diabetes_012']], data = df)
   print(summary(results))
}

                         Df Sum Sq Mean Sq F value Pr(>F)    
df[["Diabetes_012"]]      1  11384   11384   13449 <2e-16 ***
Residuals            253678 214726       1                   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
                         Df  Sum Sq Mean Sq F value Pr(>F)    
df[["Diabetes_012"]]      1   66575   66575    8136 <2e-16 ***
Residuals            253678 2075679       8                   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
                         Df  Sum Sq Mean Sq F value Pr(>F)    
df[["Diabetes_012"]]      1   18830   18830    1378 <2e-16 ***
Residuals            253678 3466104      14                   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [26]:
for (var in vars_quantitative) {
   results <- kruskal_test(df, var, 'Diabetes_012')
   print(results)
}


	Kruskal-Wallis rank sum test

data:  df[[var1]] by df[[var2]]
Kruskal-Wallis chi-squared = 14131, df = 2, p-value < 2.2e-16


	Kruskal-Wallis rank sum test

data:  df[[var1]] by df[[var2]]
Kruskal-Wallis chi-squared = 6661.9, df = 2, p-value < 2.2e-16


	Kruskal-Wallis rank sum test

data:  df[[var1]] by df[[var2]]
Kruskal-Wallis chi-squared = 528.91, df = 2, p-value < 2.2e-16



#### Biến giải thích với nhau

In [37]:
df_category <- df %>% select(-BMI, -PhysHlth, -MentHlth)

# chuyển Diabetes_012 xuống cuối cùng
df_category <- df_category %>% select(-Diabetes_012, Diabetes_012)

chi_square_results <- data.frame(variable1 = character(), 
                                variable2 = character(), 
                                No_Diabetes = numeric(),
                                Pre_Diabetes = numeric(),
                                Diabetes = numeric(),
                                reject = logical(), 
                                stringsAsFactors = FALSE)

for (i in 1:(ncol(df_category)-1)) {
    for (j in (i+1):(ncol(df_category)-1)) {
        if (i != j && i != ncol(df_category)-1 && j != ncol(df_category)-1) {
            var1 <- colnames(df_category)[i]
            var2 <- colnames(df_category)[j]
            observes <- table(df_category[[var1]], df_category[[var2]], df_category$Diabetes_012)

            no_diabetes <- chisq.test(observes[,,1],simulate.p.value = TRUE, B = 5000)
            pre_diabetes <- chisq.test(observes[,,2],simulate.p.value = TRUE, B = 5000)
            diabetes <- chisq.test(observes[,,3],simulate.p.value = TRUE, B = 5000)
            if (no_diabetes$p.value < 0.05 && pre_diabetes$p.value < 0.05 && diabetes$p.value < 0.05) {
                reject <- TRUE
            } else {
                reject <- FALSE
            }  

            chi_square_results <- rbind(chi_square_results, data.frame(variable1 = var1, 
                                                                    variable2 = var2, 
                                                                    No_Diabetes = no_diabetes$p.value,
                                                                    Pre_Diabetes = pre_diabetes$p.value,
                                                                    Diabetes = diabetes$p.value, 
                                                                    reject = reject))
        }
    }
}
chi_square_results %>% filter(reject == TRUE)

variable1,variable2,No_Diabetes,Pre_Diabetes,Diabetes,reject
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>
HighBP,HighChol,0.00019996,0.00019996,0.00019996,TRUE
HighBP,CholCheck,0.00019996,0.00059988,0.00019996,TRUE
HighBP,Smoker,0.00019996,0.00059988,0.00019996,TRUE
HighBP,Stroke,0.00019996,0.00019996,0.00019996,TRUE
HighBP,HeartDiseaseorAttack,0.00019996,0.00019996,0.00019996,TRUE
HighBP,PhysActivity,0.00019996,0.00019996,0.00019996,TRUE
HighBP,Veggies,0.00019996,0.00219956,0.00019996,TRUE
HighBP,AnyHealthcare,0.00019996,0.00199960,0.00159968,TRUE
HighBP,GenHlth,0.00019996,0.00019996,0.00019996,TRUE


In [38]:

chi_square_results %>% filter(reject == FALSE)

variable1,variable2,No_Diabetes,Pre_Diabetes,Diabetes,reject
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>
HighBP,Fruits,0.00019996,0.09118176,0.10897820,FALSE
HighBP,HvyAlcoholConsump,0.00019996,0.14237153,0.90861828,FALSE
HighBP,NoDocbcCost,0.00059988,0.05498900,0.01979604,FALSE
HighBP,Sex,0.00019996,0.11257748,0.24275145,FALSE
HighChol,PhysActivity,0.00019996,0.16596681,0.00019996,FALSE
HighChol,HvyAlcoholConsump,0.98580284,0.35452909,0.50149970,FALSE
HighChol,AnyHealthcare,0.00019996,0.46810638,0.23595281,FALSE
HighChol,NoDocbcCost,0.75704859,0.65226955,0.00019996,FALSE
HighChol,Sex,0.00019996,0.51469706,0.00019996,FALSE


In [10]:
# kruskal-wallis results
kruskal_wallis_results <- data.frame(variable1 = character(), 
                                     variable2 = character(), 
                                     No_Diabetes = numeric(),
                                     Pre_Diabetes = numeric(),
                                     Diabetes = numeric(),
                                     reject = logical(), 
                                     stringsAsFactors = FALSE)

for (var1 in vars_quantitative) {
    for (var2 in vars_quantitative) {
        if (var1 != var2) {
            no_diabetes <- kruskal_test(df %>% filter(Diabetes_012 == 0), var1, var2)
            pre_diabetes <- kruskal_test(df %>% filter(Diabetes_012 == 1), var1, var2)
            diabetes <- kruskal_test(df %>% filter(Diabetes_012 == 2), var1, var2)
            
            if (no_diabetes$p.value < 0.05 && pre_diabetes$p.value < 0.05 && diabetes$p.value < 0.05) {
                reject <- TRUE
            } else {
                reject <- FALSE
            }  

            kruskal_wallis_results <- rbind(kruskal_wallis_results, data.frame(variable1 = var1, 
                                                                              variable2 = var2, 
                                                                              No_Diabetes = no_diabetes$p.value,
                                                                              Pre_Diabetes = pre_diabetes$p.value,
                                                                              Diabetes = diabetes$p.value, 
                                                                              reject = reject))
        }
    }
}


In [13]:
kruskal_wallis_results

variable1,variable2,No_Diabetes,Pre_Diabetes,Diabetes,reject
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>
BMI,PhysHlth,3.791959e-251,3.325823e-05,3.474879e-107,TRUE
BMI,MentHlth,1.940661e-100,1.704771e-03,1.918221e-79,TRUE
PhysHlth,BMI,0.000000e+00,7.216848e-09,1.854823e-158,TRUE
PhysHlth,MentHlth,0.000000e+00,8.495876e-139,0.000000e+00,TRUE
MentHlth,BMI,0.000000e+00,4.820994e-05,4.896501e-105,TRUE
MentHlth,PhysHlth,0.000000e+00,2.550670e-132,0.000000e+00,TRUE
